In [0]:
from pyspark.sql import SparkSession

In [0]:
sc = SparkSession.builder.appName('Ops').getOrCreate()

In [0]:
import pyspark.sql.functions as f

trials = spark.read.options(delimiter = "|", header = True, inferSchema = True).csv("/FileStore/tables/clinicaltrial_2021.csv")

pharm = spark.read.options(escape = "\"", header = True, inferSchema = True).csv("/FileStore/tables/pharma.csv")

mesh = spark.read.options(escape = "\"", header = True, inferSchema = True).csv("/FileStore/tables/mesh.csv").withColumn("code", f.split(f.col("tree"), "\.")[0]).withColumn("term", f.trim("term"))

In [0]:
#Problem 1 = "The number of studies in the dataset. You must ensure that you explicitly check distinct studies."
trials.select('Id').distinct().count() 

Out[4]: 387261

In [0]:
#Problem 2 = "You should list all the types (as contained in the Type column) of studies in the dataset along with the frequencies of each type. These should be ordered from most frequent to least frequent."
trials.dropna(subset = "Type").groupBy("Type").count().orderBy("count", ascending = False).show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
#Problem 3 = "The top 5 conditions (from Conditions) with their frequencies."
conds = trials.select('Conditions').dropna(subset = "Conditions").withColumn("temp cond", f.explode(f.split(f.col("Conditions"), ","))).groupBy(f.trim("temp cond").alias("Conditions")).count().orderBy("count", ascending = False)

conds.show(5)

+-----------------+-----+
|       Conditions|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+
only showing top 5 rows



In [0]:
#Problem 4 = "Each condition can be mapped to one or more hierarchy codes. The client wishes to know the 5 most frequent roots (i.e. the sequence of letters and numbers before the first full stop) after this is done."
conds.join(mesh, f.col("Conditions") == f.col("term"), "left").groupBy("code").sum("count").orderBy("sum(count)", ascending = False).show(5)

+----+----------+
|code|sum(count)|
+----+----------+
| C04|    143994|
| C23|    136079|
| C01|    106674|
| C14|     94523|
| C10|     92310|
+----+----------+
only showing top 5 rows



In [0]:
#Problem 5 = "Find the 10 most common sponsors that are not pharmaceutical companies, along with the number of clinical trials they have sponsored. Hint: For a basic implementation, you can assume that the Parent Company column contains all possible pharmaceutical companies." 
currentpharm = [row['Parent_Company'] for row in pharm.select("Parent_Company").distinct().collect()]

trials.filter(~f.col('Sponsor').isin(currentpharm)).groupBy(f.col("Sponsor")).count().orderBy("count", ascending = False).show(10)

+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|National Cancer I...| 3218|
|M.D. Anderson Can...| 2414|
|Assistance Publiq...| 2369|
|         Mayo Clinic| 2300|
|Merck Sharp & Doh...| 2243|
|   Assiut University| 2154|
|Novartis Pharmace...| 2088|
|Massachusetts Gen...| 1971|
|    Cairo University| 1928|
|   Hoffmann-La Roche| 1828|
+--------------------+-----+
only showing top 10 rows



In [0]:
#Problem 6 = "Plot number of completed studies each month in a given year – for the submission dataset, the year is 2021. You need to include your visualization as well as a table of all the values you have plotted for each month"

trials.filter((f.col("Status") == 'Completed') & (f.split(f.col("Completion"), " ")[1] == '2021')).groupBy(f.col("Completion")).count().orderBy("count", ascending = False).show()

+----------+-----+
|Completion|count|
+----------+-----+
|  Mar 2021| 1227|
|  Jan 2021| 1131|
|  Jun 2021| 1094|
|  May 2021|  984|
|  Apr 2021|  967|
|  Feb 2021|  934|
|  Jul 2021|  819|
|  Aug 2021|  700|
|  Sep 2021|  528|
|  Oct 2021|  187|
+----------+-----+

